In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
import matplotlib.pyplot as plt

from keras_tuner.tuners import RandomSearch,BayesianOptimization
import keras_tuner as kt
from keras_tuner.engine.hyperparameters import HyperParameters
import time
log_dur = f'{time.time()}'

In [ ]:
data_95 = pd.read_csv('project_2_dataset_95var',index_col=0)
data_99 = pd.read_csv('project_2_dataset_99var',index_col=0)
full_data_x = pd.read_csv('loan_X_train.csv',header=None)
full_data_y = pd.read_csv('loan_Y_train.csv',header=None)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sc = MinMaxScaler()
x_99_sc = sc.fit_transform(data_99.iloc[:,:-1])

In [ ]:
y_95= data_95['Target']
x_95=data_95[data_95.columns.tolist()[:-1]]
#99 var data set
y_99= data_99['Target']
x_99=data_99[data_99.columns.tolist()[:-1]]

In [ ]:
pd.DataFrame(x_99_sc)

In [ ]:
full_data_x
full_data_y

# Train-Test Split

In [ ]:
x_train_95,x_test_95,y_train_95,y_test_95 = train_test_split(x_95,y_95,test_size=.2)

In [ ]:
x_train_99,x_test_99,y_train_99,y_test_99 = train_test_split(x_99,y_99,test_size=.15)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(full_data_x,full_data_y,test_size=.2)

In [ ]:
x_train_99_sc,x_test_99_sc,y_train_99,y_test_99 = train_test_split(x_99_sc,y_99,test_size=.15)

# Model

In [ ]:
def define_model(hp = HyperParameters()):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_1',8,256,16),activation=hp.Choice('activation_1',['relu','sigmoid'])))
    model.add(Dropout(hp.Float('drop_1',0,.4,.05)))
    for i in range(hp.Int('n_layers',1,10)):
        model.add(Dense(units=hp.Int(f'units_m{i}',8,256,8),activation=hp.Choice(f'activation_middlelayer_{i}',['relu','sigmoid'])))
        if hp.Boolean(f'dropout_layer{i}')==True:
            model.add(Dropout(hp.Float(f'drop_m{i}',.2,.4,.05)))        
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")) ,loss = 'binary_crossentropy',metrics=['accuracy'])
    return model

## Parameters to Optimize

In [ ]:
tuner = BayesianOptimization(define_model,objective='val_accuracy',executions_per_trial=1,directory = 'test_7',overwrite = True,beta=3.2)
tuner.search_space_summary()

In [ ]:
tuner.search(x_train_99_sc,y_train_99,epochs = 8, batch_size = 256,validation_split = .15)

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.build(input_shape=(None,x_train_99_sc.shape[1]))

In [ ]:
print(tuner.get_best_hyperparameters()[0].values)
# print(tuner.results_summary())
best_model.summary()
# best_hp=pd.DataFrame(tuner.get_best_hyperparameters()[0].values,index=[1])
# best_hp.to_csv('best_hp.csv')

## Fitting Best model to data

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience = 30,restore_best_weights = True)
best_model.fit(x_train_99_sc,y_train_99, epochs=128,batch_size=256, validation_split=.15, callbacks=[callbacks])

## Model Diagnostics

In [ ]:
plt.plot(best_model.history.history['val_accuracy'],label='Val_Accuracy')
plt.plot(best_model.history.history['accuracy'],label='Train_Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

In [ ]:
plt.plot(best_model.history.history['val_loss'],label='Val_Loss')
plt.plot(best_model.history.history['loss'],label='Train_Loss')
plt.legend()
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

## Out of Sample

In [ ]:
tf.version.VERSION

In [ ]:
testing =load_model('yekelchik_best_my_model.h5')

In [ ]:
y_pred=(best_model.predict(x_test_99_sc) > .05).astype('int32')

In [ ]:
print(confusion_matrix(y_pred,y_test_99))
print(classification_report(y_test_99,y_pred))

In [ ]:
best_model.evaluate(x_test_99_sc,y_test_99)

## Saving Models

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
best_model.save('yekelchik_best_my_model.h5')